Checkout [fooball-data.co notes](https://www.football-data.co.uk/notes.txt) for more details on the data collected

In [1]:
import os
import numpy as np
import pandas as pd
import utils.odd_even_goals_classification as oegc

In [92]:
# dataset = oegc.get_separated_dataset( manipulate_data=False, all_columns=True, start_year="2008", countries=["england"], \
#                           leagues=["Premier League"])
# dataset.to_csv("football_data2.csv", index=False)

In [9]:
def get_sample_dataset(dataset_path):
    return pd.read_csv(dataset_path, parse_dates=["Date"], dayfirst=True,encoding='ANSI', on_bad_lines='skip')

def extract_targets(dataset):
    """
    Extracts targets from the dataset.

    Returns
    ------
    dataset: The dataset without the targets
    y: A dataset containing the targets
    """
    y = dataset[["HG", "AG"]].copy()
    y["Odd Goals"] = (dataset["HG"] + dataset["AG"]) % 2 # 1 if total goals is odd, 0 if otherwise
    y["Draw"] = np.where(dataset["Res"] == "D", 1.0, 0.0)
    return dataset.drop(columns=["HG", "AG", "Res"]), y

In [13]:
# country = "turkey"
# season = "1516"
# filename = "futbol_ligi_1.csv"
# dataset_path = os.path.join("Datasets", "football_results",  country, season, filename)
# x2 = get_sample_dataset(dataset_path)

C:\Users\USER\AppData\Local\Temp\ipykernel_5636\1034738241.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(dataset_path, parse_dates=["Date"], dayfirst=True,encoding='ANSI', on_bad_lines='skip')


In [4]:
# main_folder_path = os.path.join("Datasets", "football_results")
# scrape_football_data(main_folder_path, update_only=True, seasons=["2324"], countries="all

In [5]:
dataset = pd.read_csv("football_data.csv", parse_dates=["Date"])

In [7]:
dataset.info()
# AvgH, AvgD, AvgA are average odds for a home win, a draw and an away win respectively.
# check the get_datasets functions to see how these are computed.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225964 entries, 0 to 225963
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   Date     225719 non-null  datetime64[ns]
 1   Country  225964 non-null  object        
 2   League   225964 non-null  object        
 3   Season   225964 non-null  int64         
 4   Home     225719 non-null  object        
 5   Away     225719 non-null  object        
 6   HG       225716 non-null  float64       
 7   AG       225716 non-null  float64       
 8   Res      225716 non-null  object        
 9   AvgH     220908 non-null  float64       
 10  AvgD     220908 non-null  float64       
 11  AvgA     220904 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(5)
memory usage: 20.7+ MB


In [8]:
dataset = dataset.dropna().reset_index()

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X, y = extract_targets(dataset)

# let's treat the data as a time series. This is because my initial assumption is that past
# results may have some influence on present results. For this reason, I am not shuffling.
# However, our model may need to have an idea of current standings hence there might still
# be a need to shuffle
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, shuffle=False)

### EDA

In [11]:
train_dataset = X_train.join(y_train)

In [12]:
train_dataset

,index,Date,Country,League,Season,Home,Away,AvgH,AvgD,AvgA,HG,AG,Odd Goals,Draw
0,0,2000-07-28,France,Le Championnat,2000,Marseille,Troyes,1.490,3.500,5.250,3.0,1.0,0.0,0.0
1,1,2000-07-28,France,Le Championnat,2000,Paris SG,Strasbourg,1.375,3.675,6.750,3.0,1.0,0.0,0.0
2,6,2000-07-29,Scotland,Premier League,2000,St Mirren,Kilmarnock,2.340,3.050,2.565,0.0,1.0,1.0,0.0
3,7,2000-07-29,Scotland,Premier League,2000,Rangers,St Johnstone,1.200,4.975,10.000,2.0,1.0,1.0,0.0
4,8,2000-07-29,Scotland,Premier League,2000,Motherwell,Dundee,1.800,3.125,3.750,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150207,154807,2017-10-15,Norway,Eliteserien,2017,Kristiansund,Viking,1.930,3.560,3.800,0.0,2.0,0.0,0.0
150208,154808,2017-10-15,Sweden,Allsvenskan,2017,Elfsborg,Sundsvall,1.790,3.870,4.130,2.0,1.0,1.0,0.0
150209,154809,2017-10-15,Sweden,Allsvenskan,2017,AIK,Jonkopings,1.300,5.240,10.090,2.0,0.0,0.0,0.0
150210,154810,2017-10-15,Mexico,Liga MX,2017,Club Leon,U.A.N.L.- Tigres,2.630,3.320,2.520,1.0,0.0,1.0,0.0


In [13]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150212 entries, 0 to 150211
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   index      150212 non-null  int64         
 1   Date       150212 non-null  datetime64[ns]
 2   Country    150212 non-null  object        
 3   League     150212 non-null  object        
 4   Season     150212 non-null  int64         
 5   Home       150212 non-null  object        
 6   Away       150212 non-null  object        
 7   AvgH       150212 non-null  float64       
 8   AvgD       150212 non-null  float64       
 9   AvgA       150212 non-null  float64       
 10  HG         150212 non-null  float64       
 11  AG         150212 non-null  float64       
 12  Odd Goals  150212 non-null  float64       
 13  Draw       150212 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(2), object(4)
memory usage: 21.2+ MB


In [14]:
train_dataset.describe()

,index,Date,Season,AvgH,AvgD,AvgA,HG,AG,Odd Goals,Draw
count,150212.000000,150212,150212.000000,150212.000000,150212.000000,150212.000000,150212.000000,150212.000000,150212.000000,150212.000000
mean,78718.540509,2010-04-28 02:00:00.191729152,2009.372633,2.300636,3.386774,3.803967,1.490234,1.123093,0.487651,0.267608
min,0.000000,2000-07-28 00:00:00,2000.000000,1.010000,1.450000,1.025000,0.000000,0.000000,0.000000,0.000000
25%,40820.750000,2006-02-08 00:00:00,2005.000000,1.740000,3.100000,2.600000,1.000000,0.000000,0.000000,0.000000
50%,79111.500000,2011-01-29 00:00:00,2010.000000,2.100000,3.225000,3.225000,1.000000,1.000000,0.000000,0.000000
75%,116998.250000,2014-09-20 00:00:00,2014.000000,2.500000,3.450000,4.300000,2.000000,2.000000,1.000000,1.000000
max,154811.000000,2017-10-15 00:00:00,2017.000000,26.650000,22.550000,62.000000,10.000000,9.000000,1.000000,1.000000
std,44287.255736,NaN,5.027917,1.066420,0.622174,2.151734,1.256242,1.099112,0.499849,0.442714


In [15]:
train_dataset[["Country", "League", "Season", "Home", "Away",]].describe()

,Season
count,150212.000000
mean,2009.372633
std,5.027917
min,2000.000000
25%,2005.000000
50%,2010.000000
75%,2014.000000
max,2017.000000


In [16]:
train_dataset

,index,Date,Country,League,Season,Home,Away,AvgH,AvgD,AvgA,HG,AG,Odd Goals,Draw
0,0,2000-07-28,France,Le Championnat,2000,Marseille,Troyes,1.490,3.500,5.250,3.0,1.0,0.0,0.0
1,1,2000-07-28,France,Le Championnat,2000,Paris SG,Strasbourg,1.375,3.675,6.750,3.0,1.0,0.0,0.0
2,6,2000-07-29,Scotland,Premier League,2000,St Mirren,Kilmarnock,2.340,3.050,2.565,0.0,1.0,1.0,0.0
3,7,2000-07-29,Scotland,Premier League,2000,Rangers,St Johnstone,1.200,4.975,10.000,2.0,1.0,1.0,0.0
4,8,2000-07-29,Scotland,Premier League,2000,Motherwell,Dundee,1.800,3.125,3.750,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150207,154807,2017-10-15,Norway,Eliteserien,2017,Kristiansund,Viking,1.930,3.560,3.800,0.0,2.0,0.0,0.0
150208,154808,2017-10-15,Sweden,Allsvenskan,2017,Elfsborg,Sundsvall,1.790,3.870,4.130,2.0,1.0,1.0,0.0
150209,154809,2017-10-15,Sweden,Allsvenskan,2017,AIK,Jonkopings,1.300,5.240,10.090,2.0,0.0,0.0,0.0
150210,154810,2017-10-15,Mexico,Liga MX,2017,Club Leon,U.A.N.L.- Tigres,2.630,3.320,2.520,1.0,0.0,1.0,0.0


In [17]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [18]:
train_dataset.head(3)

,index,Date,Country,League,Season,Home,Away,AvgH,AvgD,AvgA,HG,AG,Odd Goals,Draw
0,0,2000-07-28,France,Le Championnat,2000,Marseille,Troyes,1.490,3.500,5.250,3.0,1.0,0.0,0.0
1,1,2000-07-28,France,Le Championnat,2000,Paris SG,Strasbourg,1.375,3.675,6.750,3.0,1.0,0.0,0.0
2,6,2000-07-29,Scotland,Premier League,2000,St Mirren,Kilmarnock,2.340,3.050,2.565,0.0,1.0,1.0,0.0


In [19]:
def main_preparation(df):
    # Add three letters of country to league to differentiate leagues of same names in different countries
    dataset = df.copy()
    dataset["League"] =  (dataset["Country"].str[:3].str.strip() + " " + dataset["League"].str.strip())
    dataset["Country"] = dataset["Country"].str.strip()
    dataset["Home"] = dataset["Home"].str.strip()
    dataset["Away"] = dataset["Away"].str.strip()
    dataset["Season"] = dataset["Season"].astype(np.uint16)
    dataset["day"] = dataset["Date"].dt.day
    dataset["dayofweek"] = dataset["Date"].dt.day_of_week
    dataset["week"] = dataset["Date"].dt.isocalendar()["week"]
    dataset["month"] = dataset["Date"].dt.month
    dataset["quarter"] = dataset["Date"].dt.quarter
    dataset["year"] = dataset["Date"].dt.year
    return dataset.drop(columns=["Date", "index"])

In [20]:
# Frequency encoding: Use a function transformer for this
#Fit
home_freq = X_train["Home"].value_counts() / len(X_train)
away_freq = X_train["Away"].value_counts() / len(X_train)

# Transform Train
X_train["Home_enc"] = X_train["Home"].apply(lambda x: home_freq[x] if x in home_freq.index else 0 )
X_train["Away_enc"] = X_train["Away"].apply(lambda x: away_freq[x] if x in away_freq.index else 0 )

# Transform Val
X_val["Home_enc"] =  X_val["Home"].apply(lambda x: home_freq[x] if x in home_freq.index else 0 )
X_val["Away_enc"] = X_val["Away"].apply(lambda x: away_freq[x] if x in away_freq.index else 0 )


In [25]:
from category_encoders import HashingEncoder

In [36]:
X_main = main_preparation(X_train)

In [51]:
column_transformer = ColumnTransformer([
    ("cat_attribs", OneHotEncoder(handle_unknown="ignore"), ["Country", "League"]), #, "Home", "Away"
    ("cat_hash1", HashingEncoder(n_components=16), ["Home", "Away"]),
], remainder="passthrough")

In [52]:
X_prepared = column_transformer.fit_transform(X_main)

### Modeling

In [53]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [54]:
logit = LogisticRegression()
sgd_clf = SGDClassifier()
tree_clf = DecisionTreeClassifier(max_depth=10, min_samples_split=10,
                                  max_features="log2", )
rf_clf = RandomForestClassifier(n_estimators=20, max_depth=25)

In [55]:
y_odd = y_train["Odd Goals"]

In [57]:
tree_clf.fit(X_prepared, y_odd)

DecisionTreeClassifier(max_depth=10, max_features='log2', min_samples_split=10)

In [60]:
column_transformer.named_transformers_.cat_hash1.feature_names_out_, column_transformer.named_transformers_.cat_attribs.categories_

(['col_0',
  'col_1',
  'col_2',
  'col_3',
  'col_4',
  'col_5',
  'col_6',
  'col_7',
  'col_8',
  'col_9',
  'col_10',
  'col_11',
  'col_12',
  'col_13',
  'col_14',
  'col_15'],
 [array(['Argentina', 'Austria', 'Belgium', 'Brazil', 'China', 'Denmark',
         'England', 'Finland', 'France', 'Germany', 'Greece', 'Ireland',
         'Italy', 'Japan', 'Mexico', 'Netherlands', 'Norway', 'Poland',
         'Portugal', 'Romania', 'Russia', 'Scotland', 'Spain', 'Sweden',
         'Switzerland', 'Turkey', 'USA'], dtype=object),
  array(['Arg Liga Profesional', 'Aus Bundesliga', 'Bel Jupiler League',
         'Bra Serie A', 'Chi Super League', 'Den Superliga',
         'Eng Championship', 'Eng Conference', 'Eng League 1',
         'Eng League 2', 'Eng Premier League', 'Fin Veikkausliiga',
         'Fra Division 2', 'Fra Le Championnat', 'Ger Bundesliga 1',
         'Ger Bundesliga 2', 'Gre Ethniki Katigoria',
         'Ire Premier Division', 'Ita Serie A', 'Ita Serie B',
         'Jap J1 

In [71]:
feature_names = ['Argentina', 'Austria', 'Belgium', 'Brazil', 'China', 'Denmark', 'England', 'Finland', 'France', 'Germany',\
                 'Greece', 'Ireland', 'Italy', 'Japan', 'Mexico', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania',\
                 'Russia', 'Scotland', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'USA', 'Arg Liga Profesional',\
                 'Aus Bundesliga', 'Bel Jupiler League', 'Bra Serie A', 'Chi Super League', 'Den Superliga', 'Eng Championship',\
                 'Eng Conference', 'Eng League 1', 'Eng League 2', 'Eng Premier League', 'Fin Veikkausliiga', 'Fra Division 2',\
                 'Fra Le Championnat', 'Ger Bundesliga 1', 'Ger Bundesliga 2', 'Gre Ethniki Katigoria', 'Ire Premier Division',\
                 'Ita Serie A', 'Ita Serie B', 'Jap J1 League', 'Mex Liga MX', 'Net Eredivisie', 'Nor Eliteserien', \
                 'Pol Ekstraklasa', 'Por Liga I', 'Rom Liga 1', 'Rus Premier League', 'Sco Division 1', 'Sco Division 2',\
                 'Sco Division 3', 'Sco Premier League', 'Spa La Liga Primera Division', 'Spa La Liga Segunda Division',\
                 'Swe Allsvenskan', 'Swi Super League', 'Tur Futbol Ligi 1', 'USA MLS', 'col_0', 'col_1', 'col_2', 'col_3', \
                 'col_4', 'col_5', 'col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14', \
                 'col_15', 'Season', 'AvgH', 'AvgD', 'AvgA', 'Home_enc', 'Away_enc', 'day', 'dayofweek', 'week', 'month', \
                 'quarter', 'year']

export_graphviz(
 tree_clf,
 out_file="images/odd_tree.dot",
 feature_names=feature_names,
 class_names=["Even", "Odd"],
 rounded=True,
 filled=True
 )

#### Tuning the hyperparameters of an estimator
1. Exhaustive Grid Search: The grid search provided by GridSearchCV exhaustively generates candidates from a grid of parameter values specified with the param_grid parameter. For instance, the following param_grid:

** param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]**
 
specifies that two grids should be explored: one with a linear kernel and C values in [1, 10, 100, 1000], and the second one with an RBF kernel, and the cross-product of C values ranging in [1, 10, 100, 1000] and gamma values in [0.001, 0.0001].

2. Randomized Search
_* jk *_

In [ ]:
dot -Tpng iris_tree.dot -o iris_tree.png

In [81]:
!dot -Tpng images/odd_tree.dot -o images/odd_tree.png

'dot' is not recognized as an internal or external command,
operable program or batch file.


In [56]:
rf_clf.fit(X_prepared, y_odd)

RandomForestClassifier(max_depth=25, n_estimators=20)

In [58]:
y_pred = rf_clf.predict(X_prepared)

In [59]:
accuracy_score(y_odd, y_pred)

0.941136527041781

In [60]:
X_val_main = main_preparation(X_val)

In [61]:
X_val_prepared = column_transformer.transform(X_val_main)

In [63]:
y_val_pred = rf_clf.predict(X_val_prepared)

In [64]:
accuracy_score(y_val["Odd Goals"], y_val_pred)

0.5014900599796296

In [ ]:
# Accuracy score on val_set is around 50%, The model may just be randomly guessing.

In [65]:
rf_clf.classes_

array([0., 1.])

In [66]:
y_val_proba = rf_clf.predict_proba(X_val_prepared)

In [69]:
y_val[:5]

,HG,AG,Odd Goals,Draw
150212,1.0,2.0,1.0,0.0
150213,3.0,0.0,1.0,0.0
150214,1.0,1.0,0.0,1.0
150215,1.0,1.0,0.0,1.0
150216,1.0,0.0,1.0,0.0


In [74]:
x = pd.DataFrame(np.c_[y_val_proba, y_val["Draw"]], columns=["0", "1", "X"])

In [82]:
x[x["1"] > 0.75]

,0,1,X
682,0.227450,0.772550,0.0
696,0.245930,0.754070,0.0
902,0.173235,0.826765,0.0
929,0.176140,0.823860,0.0
1056,0.213098,0.786902,0.0
...,...,...,...
26094,0.162551,0.837449,0.0
26269,0.248356,0.751644,0.0
26297,0.245845,0.754155,0.0
26449,0.209558,0.790442,0.0
